In [ ]:
import pandas as pd
# import geopandas as gpd
# import matplotlib.pyplot as plt
# import h3

# from shapely.geometry import Point
# from shapely.geometry import Polygon

# import contextily as ctx

#from geopy.distance import geodesic
from datetime import datetime


In [ ]:
import itertools

In [ ]:
filename_DD = '../data/nextbike/trips_Dresden with hexagons 2025-03-17_12-01-41.pkl'
filename_FB = '../data/nextbike/trips_Freiburg with hexagons 2025-03-17_12-01-41.pkl'
df_DD = pd.read_pickle(filename_DD)
df_FB = pd.read_pickle(filename_FB)

In [ ]:
mycell='871f1b559ffffff'

In [ ]:
df_DD[df_DD.hex_id_return==mycell].shape

In [ ]:
df_DD[df_DD.hex_id_rent==mycell].shape

In [ ]:
df_DD

In [ ]:
(df_DD.hex_id_rent==df_DD.hex_id_return).value_counts()

In [ ]:
(df_FB.hex_id_rent==df_FB.hex_id_return).value_counts()

In [ ]:
df_DD

In [ ]:
# MEMO atetime_hour_rent - is a floor value of datetime

In [ ]:
# df_DD['datetime_30min_rent'] = df_DD.datetime_rent.dt.floor("30min")
# df_FB['datetime_30min_rent'] = df_FB.datetime_rent.dt.floor("30min")
# df_DD['datetime_30min_return'] = df_DD.datetime_return.dt.floor("30min")
# df_FB['datetime_30min_return'] = df_FB.datetime_return.dt.floor("30min")


In [ ]:
# MEMO repeat for df_FB

In [ ]:
for i, df_tmp in  enumerate([df_DD, df_FB]):
    # df_tmp = df_DD.copy()
    all_hex_ids = set(df_tmp['hex_id_return'])
    all_hex_ids.update(df_tmp['hex_id_rent'])
    all_times = df_tmp['datetime_hour_rent'].unique()
    full_index = pd.DataFrame(itertools.product(all_hex_ids, all_times), columns=['hex_id', 'datetime_hour'])
    trip_counts_rent = df_tmp.groupby(['hex_id_rent', 'datetime_hour_rent']).size().reset_index(name='rent_count')
    trip_counts_return = df_tmp.groupby(['hex_id_return', 'datetime_hour_return']).size().reset_index(name='return_count')
    trip_counts_rent = trip_counts_rent.rename(columns={'hex_id_rent':'hex_id', 'datetime_hour_rent':'datetime_hour'})

    trip_counts_return = trip_counts_return.rename(columns={'hex_id_return':'hex_id', 'datetime_hour_return':'datetime_hour'})

    full_index = full_index.merge(trip_counts_rent, how='left', on=['hex_id', 'datetime_hour']).fillna(0)
    full_index = full_index.merge(trip_counts_return, how='left', on=['hex_id', 'datetime_hour']).fillna(0)
    # full_index
    (full_index.rent_count==0).value_counts(normalize=True)
    (full_index.return_count==0).value_counts(normalize=True)
    grouping_rent = full_index.groupby('hex_id').rent_count.sum().sort_values(ascending=False)
    grouping_return = full_index.groupby('hex_id').return_count.sum().sort_values(ascending=False)
    borderline = 100
    len(grouping_rent)
    len(grouping_rent[grouping_rent>borderline])
    len(grouping_rent[grouping_return>borderline])
    previous_size = sum(grouping_rent)
    previous_size
    sum(grouping_return)
    # sum(grouping[grouping>borderline])/sum(grouping)
    hoo_small_hexagons_rent = grouping_rent[grouping_rent<=borderline].index
    hoo_small_hexagons_return = grouping_return[grouping_return<=borderline].index
    full_index = full_index[~full_index.hex_id.isin(hoo_small_hexagons_rent)]
    full_index = full_index[~full_index.hex_id.isin(hoo_small_hexagons_return)]
    final_size = full_index.rent_count.sum()
    final_size
    final_size - previous_size
    full_index.hex_id.nunique()
    (full_index.rent_count==0).value_counts(normalize=True)
    (full_index.return_count==0).value_counts(normalize=True)
    if i==0:
        full_index_DD = full_index.copy()
    else:
        full_index_FB = full_index.copy()

In [ ]:
# full_index_DD

In [ ]:
# full_index_DD.loc[full_index_DD.hex_id=="871f1b460ffffff", ['datetime_hour', 'rent_count', 'return_count']].plot(x='datetime_hour', y=['rent_count', 'return_count'])

In [ ]:
# full_index_DD.loc[(full_index_DD.rent_count==0)]

In [ ]:
full_index_DD.rent_count.sum()

In [ ]:
full_index_FB.rent_count.sum()

In [ ]:
full_index_FB.loc[full_index_FB.hex_id=="871f81c99ffffff", ['datetime_hour', 'rent_count', 'return_count']].plot(x='datetime_hour', y=['rent_count', 'return_count'])

In [ ]:
full_index_FB

In [ ]:
full_index_FB.loc[(full_index_FB.hex_id=="871f81c99ffffff") & (full_index_FB.datetime_hour<pd.to_datetime("2024-01-01")), ['datetime_hour', 'rent_count', 'return_count']].plot(x='datetime_hour', y=['rent_count', 'return_count'])

In [ ]:
full_index_DD.rent_count.sum()

# save

In [ ]:
time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
full_index_DD.to_pickle(f'../data/nextbike/hourly_demand_supply_Dresden {time}.pkl')
full_index_FB.to_pickle(f'../data/nextbike/hourly_demand_supply_Freiburg {time}.pkl')
full_index_DD.to_csv(f'../data/nextbike/hourly_demand_supply_Dresden {time}.csv', index=None)
full_index_FB.to_csv(f'../data/nextbike/hourly_demand_supply_Freiburg {time}.csv', index=None)

In [ ]:
time